In [ ]:
%pip install pymupdf pypdf langchain_community

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('data/2040_seoul_plan.pdf')
data_seoul = loader.load()
print(data_seoul)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(data_seoul)

for i, split in enumerate(all_splits):
    print(f'split {i+1}: -----------------')
    print(split)

In [ ]:
print(type(all_splits[0]))

In [ ]:
print(all_splits[50].page_content)
print(': -----------------: -----------------: -----------------')
print(all_splits[51].page_content)

In [ ]:
%pip install langchain_chroma

In [ ]:
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
import os
load_dotenv()

OPEN_AI_KEY = os.getenv('DEV_OPENAI_API_KEY')
embedding = OpenAIEmbeddings(model='text-embedding-3-large',api_key=OPEN_AI_KEY)
v = embedding.embed_query("뉴욕의 온실가스 저감 정책은 뭐야")
print(v)
print(len(v))

In [ ]:
from langchain_chroma import Chroma
import os 

persis_directory = './chroma_store'

if not os.path.exists(persis_directory):
    print("크로마 스토어 생성")
    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embedding,
        persist_directory=persis_directory,
    )
else :
    print("load existing chroma store")
    vectorstore = Chroma(
        embedding_function=embedding,
        persist_directory=persis_directory,
    )

In [ ]:
retriever = vectorstore.as_retriever(k=3)
docs = retriever.invoke("서울시의 환경 정책이 궁금해")

for d in docs:
    print(d, '\n\n')
    

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()

OPEN_AI_KEY = os.getenv('DEV_OPENAI_API_KEY')

chat = ChatOpenAI(model='gpt-4o-mini',api_key=OPEN_AI_KEY)

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자의 질문에 대해 아래 context에 기반하여 답변하라.:\n\n{context}"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

document_chain = create_stuff_documents_chain(chat,question_answering_prompt)

In [7]:
from langchain.memory import ChatMessageHistory

chat_history = ChatMessageHistory()
chat_history.add_user_message("서울시의 온실가스 저감 정책에 대해. 알려줘.")

answer = document_chain.invoke(
    {
        "messages" : chat_history.messages,
        "context" : docs
    }
)

chat_history.add_ai_message(answer)

print(answer)

서울시의 온실가스 저감 정책은 주로 탄소중립 실현을 목표로 하며, 여러 전략과 구체적인 목표로 구성되어 있습니다. 주요 내용은 다음과 같습니다:

1. **2050 탄소중립 목표**: 서울시는 2050년까지 탄소중립을 달성하기 위해 도시 인프라를 전환하는 전략을 수립하고 있습니다.

2. **건물 부문**: 건물에서 발생하는 탄소배출을 줄이기 위해 친환경 기술을 개발하고 적극적으로 적용하는 것이 중요합니다. 이는 에너지 효율을 높이는 건물 개보수 및 신축을 포함합니다.

3. **친환경 교통**: 미래 모빌리티 기술을 활용하고 친환경 수송 차량 및 관련 인프라를 확충하여 교통 부문에서의 탄소배출을 감소시키려고 합니다.

4. **청정 에너지 기반 구축**: 에너지 전환을 위한 청정에너지의 기반을 마련하고, 재생 가능 에너지의 비율을 높이도록 노력합니다.

5. **대기질 개선**: 대기 환경을 고려한 공간계획과 배출원 관리체계를 강화하여 미세먼지와 기타 대기오염 물질의 배출을 줄이는 것도 중요한 요소입니다.

6. **자원 순환 체계 구축**: 자립적인 자원 순환 체계를 마련하여 폐기물 처리와 자원 회수의 효율성을 높입니다. 분산형 폐기물 처리 시설을 구축하고 시민들의 기후 행동을 활성화하는 거버넌스를 구축할 계획입니다.

7. **녹지 공간 조성**: 도심 숲 및 생활 공원 등을 조성하여 시민들이 쾌적한 환경을 누릴 수 있도록 하며, 이는 자연환경 보존에도 기여합니다.

8. **목표 달성 지표**: 서울시는 2005년 대비 70%의 온실가스를 감축하는 목표를 설정하고 있으며, 다양한 에너지 효율화 사업과 신재생에너지 보급 등 여러 지표에 따라 진행 상황을 평가하고 관리할 계획입니다.

서울시는 이러한 정책들을 통해 기후변화에 효과적으로 대응하고, 도시 환경을 개선하며, 시민의 건강을 지키기 위한 노력을 지속적으로 이어가고 있습니다.


In [8]:
for m in chat_history.messages:
    print(m)

content='서울시의 온실가스 저감 정책에 대해. 알려줘.' additional_kwargs={} response_metadata={}
content='서울시의 온실가스 저감 정책은 주로 탄소중립 실현을 목표로 하며, 여러 전략과 구체적인 목표로 구성되어 있습니다. 주요 내용은 다음과 같습니다:\n\n1. **2050 탄소중립 목표**: 서울시는 2050년까지 탄소중립을 달성하기 위해 도시 인프라를 전환하는 전략을 수립하고 있습니다.\n\n2. **건물 부문**: 건물에서 발생하는 탄소배출을 줄이기 위해 친환경 기술을 개발하고 적극적으로 적용하는 것이 중요합니다. 이는 에너지 효율을 높이는 건물 개보수 및 신축을 포함합니다.\n\n3. **친환경 교통**: 미래 모빌리티 기술을 활용하고 친환경 수송 차량 및 관련 인프라를 확충하여 교통 부문에서의 탄소배출을 감소시키려고 합니다.\n\n4. **청정 에너지 기반 구축**: 에너지 전환을 위한 청정에너지의 기반을 마련하고, 재생 가능 에너지의 비율을 높이도록 노력합니다.\n\n5. **대기질 개선**: 대기 환경을 고려한 공간계획과 배출원 관리체계를 강화하여 미세먼지와 기타 대기오염 물질의 배출을 줄이는 것도 중요한 요소입니다.\n\n6. **자원 순환 체계 구축**: 자립적인 자원 순환 체계를 마련하여 폐기물 처리와 자원 회수의 효율성을 높입니다. 분산형 폐기물 처리 시설을 구축하고 시민들의 기후 행동을 활성화하는 거버넌스를 구축할 계획입니다.\n\n7. **녹지 공간 조성**: 도심 숲 및 생활 공원 등을 조성하여 시민들이 쾌적한 환경을 누릴 수 있도록 하며, 이는 자연환경 보존에도 기여합니다.\n\n8. **목표 달성 지표**: 서울시는 2005년 대비 70%의 온실가스를 감축하는 목표를 설정하고 있으며, 다양한 에너지 효율화 사업과 신재생에너지 보급 등 여러 지표에 따라 진행 상황을 평가하고 관리할 계획입니다.\n\n서울시는 이러한 정책들을 통해 기후변화에 효과적으로 대응하고, 도시 환경을 개선하며, 시

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [10]:
query_for_nyc = "뉴욕은?"

query_augmentation_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"), #기존 대화 내용
        (
            "system",
            "기존의 대화 내용을 활용하여 사용자가 질문한 의도를 파악해서 한 문장의 명료한 질문으로 변환하라. 대명사나 이, 저 , 그와 같은 표현을 명확한 명사로 표현하라. :\n\n{query}"
        )
    ]
)

In [13]:
query_augmentation_chain = query_augmentation_prompt | chat | StrOutputParser()

In [15]:
augmented_query = query_augmentation_chain.invoke(
    {
        "messages" : chat_history.messages,
        "query" : query_for_nyc,
    }
)

print(augmented_query)

뉴욕의 온실가스 저감 정책은 무엇인가요?


In [16]:
docs = retriever.invoke(augmented_query)

for d in docs: 
    print(d, '\n\n')

page_content='정책 수립 시 주요하게 고려한다.
장기적·구조적 전환을 위해 도시계획 단계에서의 탄소중립·기후위기 적응대책 반영
Ÿ 탄소중립 및 기후위기 대책을 개별 사업 단위로 마련해 왔으나, 앞으로는 도시계획 및 도시 인프라 전
반에서 더욱 장기적이고 구조적인 접근이 필요하다. 
Ÿ 토지이용 공간구조 개편, 주거환경, 공원녹지 등 관리계획 수립 시에도 기후변화 대응에 따른 계획 
요소를 반영할 수 있도록 관련 기준을 제시한다.
Ÿ 친환경 기술 개발 및 적용을 통한 탄소 배출량 감축, 친환경 교통 인프라 확충, 지역 내 청정에너지 
기반 구축 등 탄소배출 감소를 위한 도시 에너지 체계 개편을 적극 추진한다.' metadata={'total_pages': 205, 'author': 'SI', 'producer': 'Hancom PDF 1.3.0.542', 'source': 'data/2040_seoul_plan.pdf', 'title': '', 'creationdate': '2024-12-12T18:16:11+09:00', 'trapped': '', 'creator': 'Hwp 2020 11.0.0.5178', 'moddate': '2024-12-12T18:16:11+09:00', 'keywords': '', 'subject': '', 'format': 'PDF 1.4', 'creationDate': "D:20241212181611+09'00'", 'modDate': "D:20241212181611+09'00'", 'page': 64, 'file_path': 'data/2040_seoul_plan.pdf'} 


page_content='56
제2장 미래상과 목표
6. 미래위기를 준비하는, ‘탄소중립 안전도시 구축’
1) 배경
전(全) 지구적인 기후변화에 대응하기 위한 대도시 차원의 대응 필요
Ÿ 서울시 2017년 온실가스 배출량은 46,685천 톤CO2eq로 2005년 배출량에 비해 5.6%(276만 톤
CO2eq) 감소하였으며, 서울의 1인당 온실

In [17]:
chat_history.add_user_message(query_for_nyc)

answer = document_chain.invoke(
    {"messages" : chat_history.messages,
     "context" : docs}
)

chat_history.add_ai_message(answer)

print(answer)


뉴욕시의 온실가스 저감 정책은 지속 가능한 도시 발전과 기후변화 대응에 중점을 두고 다양한 전략을 추진하고 있습니다. 주요 내용은 다음과 같습니다:

1. **2030년 탄소배출 40% 감축 목표**: 뉴욕시는 2005년 대비 2030년까지 온실가스를 40% 줄이는 것을 목표로 설정하였습니다.

2. **그린 뉴딜**: 뉴욕시는 "그린 뉴딜" 프로그램을 통해 신재생 에너지 사용 확대, 에너지 효율 개선, 녹색 일자리 창출 등을 추진하고 있습니다.

3. **건물 에너지 효율 강화**: 건물에서 발생하는 온실가스의 약 70%가 높은 에너지 소비에서 비롯되므로, 기존 건물에 대한 에너지 효율화 규제를 강화하고, 새로운 건물에 대해서는 에너지 효율 기준을 설정하고 있습니다. 건물의 에너지 성능을 개선하기 위한 '건물 에너지 법안’과 '건축물의 지속 가능성 인증 제도'를 시행하고 있습니다.

4. **전기차 및 대중교통 활성화**: 저탄소 교통 시스템으로의 전환을 위해 전기차 충전소를 늘리고, 대중교통 이용을 장려하기 위한 캠페인과 인프라 구축을 진행하고 있습니다. 또한, 전기버스와 같은 친환경 대중교통 수단 확대에도 힘쓰고 있습니다.

5. **도심 녹지 조성 및 자전거 인프라**: 도심 내 공원과 녹지 공간을 늘리고, 자전거 도로와 인프라를 확충하여 시민들이 친환경 교통수단을 이용할 수 있도록 지원합니다.

6. **재생 가능 에너지 사용 확대**: 태양광, 풍력 등 재생 가능 에너지 프로젝트에 대한 투자를 증가시켜, 전체 에너지 믹스에서 청정 에너지의 비율을 높이는 것이 목표입니다.

7. **건축물의 제로에너지 목표**: 모든 신규 건물에 대해 제로에너지 건물(ZEB)로의 전환을 의무화하는 정책을 실시하고 있습니다.

8. **시민 참여와 교육**: 시민의 기후 위기 인식을 높이고 지속 가능한 행동을 촉진하기 위해 다양한 교육 프로그램과 캠페인을 진행하고 있습니다.

이와 함께 뉴욕시는 국제적인 기후 행동의 일환으로 파리 협정의 목표를 준수하고 있으며, 전 세